# Validation and clean

# Stage 1 Validation

In [32]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

MASTER_TABLE_FILE_CSV = 'olist_master_feature_table_v1.csv'

try:
    df = pd.read_csv(MASTER_TABLE_FILE_CSV)
    print(f"Tải file CSV '{MASTER_TABLE_FILE_CSV}' thành công.")
except FileNotFoundError:
    print(f"LỖI: Không tìm thấy file checkpoint. Vui lòng chạy lại Notebook 01.")
    # Dừng ở đây nếu không có dữ liệu
    assert False, "Không tìm thấy file dữ liệu đầu vào."

print(f"\nKích thước Bảng Dữ liệu: {df.shape}")
display(df.head())

Tải file CSV 'olist_master_feature_table_v1.csv' thành công.

Kích thước Bảng Dữ liệu: (112650, 21)


,order_id,product_id,customer_id,seller_id,order_item_id,order_purchase_timestamp,delivery_time_days,delivery_vs_estimated_days,price,freight_value,freight_ratio,is_payment_credit_card,is_payment_boleto,is_payment_installments,payment_value_total,review_score,avg_review_score_product,avg_review_score_seller,product_category_name,customer_state,seller_state
0,e481f51cbdc54678b7cc49136f2d6af7,87285b34884572647811a353c7ac498a,9ef432eb6251297304e76186b10a928d,3504c0cb71d7fa48d967e0e4c94d59d9,1.00,2017-10-02 10:56:33,8.44,7.11,29.99,8.72,0.29,1,0,1,38.71,4.00,4.00,4.42,utilidades_domesticas,SP,SP
1,53cdb2fc8bc7dce0b6741e2150273451,595fac2a385ac33a80bd5114aec74eb8,b0830fb4747a6c6d20dea0b8c802d7ef,289cdb325fb7e7f891c38608bf9e0962,1.00,2018-07-24 20:41:37,13.78,5.36,118.70,22.76,0.19,0,1,0,141.46,4.00,4.41,4.58,perfumaria,BA,SP
2,47770eb9100c2d0c44946d9cf07ec65d,aa4383b373c6aca5d8797843e5594415,41ce2a54c0b03bf3443c3d931a367089,4869f7a5dfa277a7dca6462dcf3b52b2,1.00,2018-08-08 08:38:49,9.39,17.25,159.90,19.22,0.12,1,0,1,179.12,5.00,5.00,4.12,automotivo,GO,SP
3,949d5b44dbf5de918fe9c16f97b45f8a,d0b61bfb1de832b15ba9d266ca96e5b0,f88197465ea7920adcdbec7375364d82,66922902710d126a0e7d26b0e3805106,1.00,2017-11-18 19:28:06,13.21,12.98,45.00,27.20,0.60,1,0,0,72.20,5.00,4.25,4.45,pet_shop,RN,MG
4,ad21c59c0840e6cb83a9ceb5573f8159,65266b2da20d04dbe00c5c2d3bb7859e,8ab97904e6daea8866dbdbc4fb7aad2c,2c9e548be18521d1c43cde1c582c6de8,1.00,2018-02-13 21:18:39,2.87,9.24,19.90,8.72,0.44,1,0,0,28.62,5.00,3.73,3.92,papelaria,SP,SP


In [33]:
print("--- 1. Kiểm tra Dữ liệu bị thiếu (Null Check) ---")

null_counts = df.isnull().sum()

# Chỉ in ra các cột CÓ dữ liệu bị thiếu
print("Các cột có dữ liệu bị thiếu (sắp xếp giảm dần):")
null_summary = null_counts[null_counts > 0].sort_values(ascending=False)
null_summary_percent = (null_summary / len(df) * 100)

# Tạo một DataFrame đẹp để xem
null_report = pd.DataFrame({
    'Total Nulls': null_summary,
    'Percentage': null_summary_percent
})
display(null_report)

print("\n*Ghi chú (Note):*")
print("- `delivery_time_days`, `delivery_vs_estimated_days` bị thiếu là bình thường (đơn hàng chưa giao).")
print("- `review_score` bị thiếu là bình thường (đơn hàng chưa được review).")

--- 1. Kiểm tra Dữ liệu bị thiếu (Null Check) ---
Các cột có dữ liệu bị thiếu (sắp xếp giảm dần):


,Total Nulls,Percentage
delivery_time_days,2454,2.18
delivery_vs_estimated_days,2454,2.18
product_category_name,1603,1.42
review_score,942,0.84
avg_review_score_product,207,0.18
avg_review_score_seller,5,0.00
payment_value_total,3,0.00



*Ghi chú (Note):*
- `delivery_time_days`, `delivery_vs_estimated_days` bị thiếu là bình thường (đơn hàng chưa giao).
- `review_score` bị thiếu là bình thường (đơn hàng chưa được review).


In [34]:
print("\n--- 2. Kiểm tra Phân phối (Distribution Check) ---")

# Chọn các cột số học quan trọng để kiểm tra
columns_to_describe = [
    'price', 'freight_value', 'freight_ratio',
    'delivery_time_days', 'delivery_vs_estimated_days',
    'review_score', 'avg_review_score_product', 'avg_review_score_seller'
]

# Đảm bảo chỉ lấy các cột tồn tại trong DataFrame
existing_cols = [col for col in columns_to_describe if col in df.columns]

if existing_cols:
    print("Bảng thống kê mô tả (Statistic Summary):")
    display(df[existing_cols].describe())
else:
    print("Không tìm thấy các cột số học để mô tả.")

print("\n*Ghi chú (Note):*")
print("- Kiểm tra `min` của 'price': Có bằng 0 không?")
print("- Kiểm tra `min` và `max` của 'delivery_time_days': Có giá trị âm (lỗi dữ liệu) hoặc quá lớn không?")
print("- Kiểm tra `max` của 'freight_ratio': Có giá trị nào cực lớn (ví dụ: > 100) không?")


--- 2. Kiểm tra Phân phối (Distribution Check) ---
Bảng thống kê mô tả (Statistic Summary):


,price,freight_value,freight_ratio,delivery_time_days,delivery_vs_estimated_days,review_score,avg_review_score_product,avg_review_score_seller
count,112650.00,112650.00,112650.00,110196.00,110196.00,111708.00,112443.00,112645.00
mean,120.65,19.99,0.32,12.47,11.33,4.03,4.03,4.03
std,183.63,15.81,0.35,9.45,10.16,1.39,0.87,0.40
min,0.85,0.00,0.00,0.53,-188.98,1.00,1.00,1.00
25%,39.90,13.08,0.13,6.74,6.48,4.00,3.75,3.88
50%,74.99,16.26,0.23,10.18,12.05,5.00,4.14,4.06
75%,134.90,21.15,0.39,15.54,16.32,5.00,4.62,4.23
max,6735.00,409.68,26.24,209.63,146.02,5.00,5.00,5.00



*Ghi chú (Note):*
- Kiểm tra `min` của 'price': Có bằng 0 không?
- Kiểm tra `min` và `max` của 'delivery_time_days': Có giá trị âm (lỗi dữ liệu) hoặc quá lớn không?
- Kiểm tra `max` của 'freight_ratio': Có giá trị nào cực lớn (ví dụ: > 100) không?


In [35]:
print("\n--- 3. Kiểm tra Tính duy nhất (Cardinality Check) ---")

# Chúng ta kiểm tra cột 'product_category_name'
column_to_check = 'product_category_name'

if column_to_check in df.columns:
    print(f"Số lượng danh mục (category) duy nhất: {df[column_to_check].nunique()}")

    print("\nTop 5 danh mục phổ biến nhất:")
    print(df[column_to_check].value_counts().head(5))

    print("\nTop 5 danh mục ÍT phổ biến nhất (để kiểm tra rác/lỗi chính tả):")
    print(df[column_to_check].value_counts().tail(5))
else:
    print(f"Cột '{column_to_check}' không tồn tại.")

print("\n*Ghi chú (Note):*")
print("- Hãy tìm các giá trị gần giống nhau, ví dụ: 'abc_xyz' và 'abc_xyz ' (có khoảng trắng).")
print("- Đây là cơ hội để làm sạch (cleaning) và nhóm (grouping) lại các danh mục.")


--- 3. Kiểm tra Tính duy nhất (Cardinality Check) ---
Số lượng danh mục (category) duy nhất: 73

Top 5 danh mục phổ biến nhất:
product_category_name
cama_mesa_banho           11115
beleza_saude               9670
esporte_lazer              8641
moveis_decoracao           8334
informatica_acessorios     7827
Name: count, dtype: int64

Top 5 danh mục ÍT phổ biến nhất (để kiểm tra rác/lỗi chính tả):
product_category_name
cds_dvds_musicais                14
la_cuisine                       14
pc_gamer                          9
fashion_roupa_infanto_juvenil     8
seguros_e_servicos                2
Name: count, dtype: int64

*Ghi chú (Note):*
- Hãy tìm các giá trị gần giống nhau, ví dụ: 'abc_xyz' và 'abc_xyz ' (có khoảng trắng).
- Đây là cơ hội để làm sạch (cleaning) và nhóm (grouping) lại các danh mục.


In [36]:
print("\n--- 4. Kiểm tra Tính Toàn vẹn của Join (Join Integrity Check) ---")

# Đây là bài kiểm tra QUAN TRỌNG NHẤT
# Độ chi tiết (granularity) của bảng phải là [order_id + order_item_id]
# (Mỗi hàng phải là duy nhất cho 2 cột này)

key_columns = ['order_id', 'order_item_id']

# Kiểm tra xem các cột khóa có tồn tại không
if 'order_item_id' not in df.columns:
    # Một số item không có order_item_id (ví dụ: bị null),
    # chúng ta dùng (order_id, product_id) làm khóa thay thế
    key_columns = ['order_id', 'product_id']

if all(col in df.columns for col in key_columns):
    # Đếm số hàng bị trùng lặp dựa trên 2 cột khóa

    # Chúng ta phải xử lý các giá trị Nulls trong các cột khóa trước khi kiểm tra
    # (Nếu không, duplicated() sẽ coi các 'Null' là khác nhau)
    # Tạm thời điền 'Null' bằng một giá trị placeholder

    df_check = df.copy()
    df_check[key_columns[0]] = df_check[key_columns[0]].fillna('MISSING_ORDER')
    df_check[key_columns[1]] = df_check[key_columns[1]].fillna('MISSING_ITEM')

    duplicate_rows = df_check.duplicated(subset=key_columns).sum()

    print(f"Số lượng hàng bị trùng lặp (duplicate) theo khóa {key_columns}: {duplicate_rows}")

    if duplicate_rows == 0:
        print("-> ✅ TỐT! Pipeline hợp nhất (merge) an toàn. Độ chi tiết chính xác.")
        print("-> (Điều này chứng tỏ việc aggregate bảng 'payments' trước khi merge là chính xác).")
    else:
        print("-> 🚨 CẢNH BÁO! Pipeline đã tạo ra dữ liệu trùng lặp.")
        print("-> HÀNH ĐỘNG: Cần xem lại ngay lập tức logic merge ở Notebook 01.")

else:
    print(f"Không thể kiểm tra trùng lặp vì thiếu các cột khóa chính.")


--- 4. Kiểm tra Tính Toàn vẹn của Join (Join Integrity Check) ---
Số lượng hàng bị trùng lặp (duplicate) theo khóa ['order_id', 'order_item_id']: 0
-> ✅ TỐT! Pipeline hợp nhất (merge) an toàn. Độ chi tiết chính xác.
-> (Điều này chứng tỏ việc aggregate bảng 'payments' trước khi merge là chính xác).


# Stage 2 Data Cleaning

In [37]:
print("--- 2.1 Bắt đầu Xử lý Dữ liệu bị thiếu ---")

# Tạo một bản sao (copy) để làm sạch, giữ bản gốc (df) để so sánh
df_clean = df.copy()

# 1. Cột 'review_score'
# Bị thiếu vì đơn hàng chưa được review.
# Chúng ta sẽ điền (impute) = 0 để mô hình hiểu "Chưa có review".
df_clean['review_score'] = df_clean['review_score'].fillna(0)
print("Đã điền 'review_score' bị thiếu bằng 0.")

# 2. Cột 'avg_review_score_product' & 'avg_review_score_seller'
# Bị thiếu nếu sản phẩm/người bán đó là 'mới' (chưa có review nào).
# Chúng ta sẽ điền bằng điểm review trung bình của toàn bộ (khoảng 4.0)
mean_global_review = df_clean['review_score'].mean()
df_clean['avg_review_score_product'] = df_clean['avg_review_score_product'].fillna(mean_global_review)
df_clean['avg_review_score_seller'] = df_clean['avg_review_score_seller'].fillna(mean_global_review)
print(f"Đã điền 'avg_review_score...' bị thiếu bằng {mean_global_review:.2f}.")

# 3. Cột 'delivery_time_days' & 'delivery_vs_estimated_days'
# Bị thiếu vì đơn hàng chưa giao.
# Chúng ta điền một giá trị đặc biệt (ví dụ: -999) để mô hình có thể học
# đây là một nhóm riêng ("Chưa giao").
df_clean['delivery_time_days'] = df_clean['delivery_time_days'].fillna(-999)
df_clean['delivery_vs_estimated_days'] = df_clean['delivery_vs_estimated_days'].fillna(-999)
print("Đã điền 'delivery_time...' bị thiếu bằng -999 (chưa giao).")

# 4. Cột 'product_category_name'
# Bị thiếu do dữ liệu.
# Chúng ta điền bằng 'unknown' (không xác định).
df_clean['product_category_name'] = df_clean['product_category_name'].fillna('unknown')
print("Đã điền 'product_category_name' bị thiếu bằng 'unknown'.")

# 5. Các cột khác (ví dụ: payment...)
# Tạm thời điền các cột số học còn lại bằng 0
numeric_cols_with_nulls = ['payment_installments_total', 'payment_value_total']
for col in numeric_cols_with_nulls:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].fillna(0)

# Kiểm tra lại
print("\nKiểm tra lại Nulls (chỉ in cột còn thiếu):")
print(df_clean.isnull().sum()[df_clean.isnull().sum() > 0])

--- 2.1 Bắt đầu Xử lý Dữ liệu bị thiếu ---
Đã điền 'review_score' bị thiếu bằng 0.
Đã điền 'avg_review_score...' bị thiếu bằng 4.00.
Đã điền 'delivery_time...' bị thiếu bằng -999 (chưa giao).
Đã điền 'product_category_name' bị thiếu bằng 'unknown'.

Kiểm tra lại Nulls (chỉ in cột còn thiếu):
Series([], dtype: int64)


In [38]:
# %% [code]
# --- 2.2 Xử lý Tính duy nhất (Handling Cardinality/Noise) ---
print("\n--- 2.2 Bắt đầu Xử lý 'product_category_name' ---")

col_to_clean = 'product_category_name'

if col_to_clean in df_clean.columns:
    # 1. Chuyển sang chữ thường (lowercase)
    df_clean[col_to_clean] = df_clean[col_to_clean].str.lower()

    # 2. Xóa khoảng trắng thừa (trailing spaces)
    df_clean[col_to_clean] = df_clean[col_to_clean].str.strip()

    # 3. (Tùy chọn) Thay thế các ký tự đặc biệt (ví dụ: '_', ' ')
    # df_clean[col_to_clean] = df_clean[col_to_clean].str.replace('_', ' ')

    print(f"Số lượng danh mục duy nhất SAU KHI làm sạch: {df_clean[col_to_clean].nunique()}")
else:
    print(f"Cột '{col_to_clean}' không tồn tại.")


--- 2.2 Bắt đầu Xử lý 'product_category_name' ---
Số lượng danh mục duy nhất SAU KHI làm sạch: 74


In [39]:
# %% [code]
# --- 2.3 Xử lý Phân phối & Bất thường (Handling Outliers) ---
print("\n--- 2.3 Bắt đầu Xử lý Outliers ---")

# 1. Xử lý 'delivery_time_days' bị âm (lỗi dữ liệu)
# (Chúng ta loại trừ giá trị -999 mà chúng ta vừa điền vào)
negative_delivery_count = (df_clean['delivery_time_days'] < 0) & (df_clean['delivery_time_days'] != -999)
print(f"Tìm thấy {negative_delivery_count.sum()} đơn hàng có thời gian giao hàng < 0 ngày (lỗi).")

# Hành động: Cập nhật các giá trị âm này thành 0 (giao trong ngày)
df_clean.loc[negative_delivery_count, 'delivery_time_days'] = 0

# 2. Xử lý 'freight_ratio' (tỷ lệ phí ship/giá) quá lớn
# Các giá trị này xảy ra khi 'price' gần = 0.
# Chúng ta sẽ "cắt bớt" (clip) các giá trị này ở một ngưỡng (ví dụ: 10)
# (Nghĩa là, phí ship tối đa gấp 10 lần giá sản phẩm)
p_99 = df_clean['freight_ratio'].quantile(0.99)
print(f"Ngưỡng 99% của 'freight_ratio' là: {p_99:.2f}")

# Giả sử chúng ta clip ở mức 10 (hợp lý)
CLIP_VALUE = 10
count_clipped = (df_clean['freight_ratio'] > CLIP_VALUE).sum()
print(f"Đang 'cắt bớt' {count_clipped} hàng có 'freight_ratio' > {CLIP_VALUE}.")
df_clean['freight_ratio'] = df_clean['freight_ratio'].clip(upper=CLIP_VALUE)

print("\nKiểm tra lại bảng thống kê mô tả (describe) SAU KHI làm sạch:")
display(df_clean[existing_cols].describe())


--- 2.3 Bắt đầu Xử lý Outliers ---
Tìm thấy 0 đơn hàng có thời gian giao hàng < 0 ngày (lỗi).
Ngưỡng 99% của 'freight_ratio' là: 1.55
Đang 'cắt bớt' 4 hàng có 'freight_ratio' > 10.

Kiểm tra lại bảng thống kê mô tả (describe) SAU KHI làm sạch:


,price,freight_value,freight_ratio,delivery_time_days,delivery_vs_estimated_days,review_score,avg_review_score_product,avg_review_score_seller
count,112650.00,112650.00,112650.00,112650.00,112650.00,112650.00,112650.00,112650.00
mean,120.65,19.99,0.32,-9.56,-10.68,4.00,4.03,4.03
std,183.63,15.81,0.33,147.95,147.83,1.43,0.87,0.40
min,0.85,0.00,0.00,-999.00,-999.00,0.00,1.00,1.00
25%,39.90,13.08,0.13,6.36,6.25,3.00,3.75,3.88
50%,74.99,16.26,0.23,10.05,11.48,5.00,4.14,4.06
75%,134.90,21.15,0.39,15.33,16.25,5.00,4.62,4.23
max,6735.00,409.68,10.00,209.63,146.02,5.00,5.00,5.00


In [52]:
# %% [code]
# --- Giai đoạn 3: Lưu trữ Dữ liệu Đã Làm sạch (Save Clean Data - CSV Only) ---
print("\n--- Giai đoạn 3: Lưu trữ Dữ liệu Đã Làm sạch ---")

# 1. Kiểm tra lại lần cuối
print(f"Số hàng của bảng gốc (raw): {df.shape[0]}")
print(f"Số hàng của bảng sạch (clean): {df_clean.shape[0]}")
print("Kiểm tra Nulls cuối cùng (phải bằng 0 hoặc rất gần 0):")
print(df_clean.isnull().sum().sum())

# 2. Định nghĩa tên file CSV
CLEAN_OUTPUT_FILE_CSV = 'olist_master_table_CLEAN_v1.csv'

# 3. Lưu file đã làm sạch (Chỉ CSV)
print(f"\nĐang lưu file {CLEAN_OUTPUT_FILE_CSV} (việc này có thể mất vài phút)...")
try:
    # index=False là RẤT QUAN TRỌNG để không lưu chỉ số (index) của Pandas
    df_clean.to_csv(CLEAN_OUTPUT_FILE_CSV, index=False)

    print(f"\nĐã lưu file CSV sạch tại: {CLEAN_OUTPUT_FILE_CSV}")
    print("\n--- 🥳 HOÀN THÀNH NOTEBOOK 02 (CLEANING) ---")

except Exception as e:
    print(f"\n🚨 LỖI: Đã xảy ra lỗi khi cố gắng lưu file CSV.")
    print(f"Chi tiết lỗi: {e}")


--- Giai đoạn 3: Lưu trữ Dữ liệu Đã Làm sạch ---
Số hàng của bảng gốc (raw): 112650
Số hàng của bảng sạch (clean): 112650
Kiểm tra Nulls cuối cùng (phải bằng 0 hoặc rất gần 0):
0

Đang lưu file olist_master_table_CLEAN_v1.csv (việc này có thể mất vài phút)...

Đã lưu file CSV sạch tại: olist_master_table_CLEAN_v1.csv

--- 🥳 HOÀN THÀNH NOTEBOOK 02 (CLEANING) ---


# 3 Revalidation

In [53]:
print("--- 🩺 BẮT ĐẦU TÁI THẨM ĐỊNH (RE-VALIDATION) TRÊN df_clean ---")
print("=" * 60)

# Giả định df (bản gốc) và df_clean (đã làm sạch) đều tồn tại

# --- 1. TÁI KIỂM TRA: Dữ liệu bị thiếu (Null Check) ---
print("--- 1. TÁI KIỂM TRA: Dữ liệu bị thiếu (Null Check) ---")
final_nulls = df_clean.isnull().sum().sum()
print(f"Tổng số Nulls còn lại trong toàn bộ DataFrame: {final_nulls}")

if final_nulls == 0:
    print("-> ✅ TỐT! Đã dọn sạch (impute) tất cả Nulls.")
else:
    print(f"-> 🚨 CẢNH BÁO! Vẫn còn {final_nulls} giá trị Null chưa được xử lý.")
print("-" * 60)


# --- 2. TÁI KIỂM TRA: Phân phối (Distribution Check) ---
print("--- 2. TÁI KIỂM TRA: Phân phối (Distribution Check) ---")
# Lọc (filter) ra các hàng không phải là -999 (đã giao)
delivered_mask = df_clean['delivery_time_days'] != -999
min_real = df_clean[delivered_mask]['delivery_time_days'].min()

print(f"Min thực tế của 'delivery_time_days' (đã giao): {min_real:.4f}")

# Chúng ta kiểm tra xem nó có >= 0 (lớn hơn hoặc bằng 0) hay không
if min_real >= 0:
    print("-> ✅ TỐT! Đã làm sạch thành công. Không còn giá trị âm (lỗi).")
else:
    print(f"-> 🚨 LỖI! Vẫn còn giá trị âm (lỗi dữ liệu thực sự): {min_real}")
print("-" * 60)


# --- 3. TÁI KIỂM TRA: Tính duy nhất (Cardinality Check) ---
print("--- 3. TÁI KIỂM TRA: Tính duy nhất (Cardinality Check) ---")
col_to_check = 'product_category_name'

# Lấy số 'nunique' *trước* (từ df, không tính NaN)
count_before_clean = df[col_to_check].dropna().nunique()

# Lấy số 'nunique' *sau* (từ df_clean, không tính 'unknown')
count_after_clean = df_clean[df_clean[col_to_check] != 'unknown'][col_to_check].nunique()

print(f"Số danh mục (TRƯỚC): {count_before_clean} (không tính Null)")
print(f"Số danh mục (SAU): {count_after_clean} (không tính 'unknown')")

if count_after_clean <= count_before_clean:
    print(f"-> ✅ TỐT! Các lệnh .lower()/.strip() đã hoạt động (hoặc dữ liệu vốn đã sạch).")
else:
    print("-> 🚨 LỖI! Số danh mục đã tăng lên (lỗi logic).")
print("-" * 60)


# --- 4. TÁI KIỂM TRA: Tính Toàn vẹn của Join (Integrity Check) ---
print("--- 4. TÁI KIỂM TRA: Tính Toàn vẹn của Join (Integrity Check) ---")
key_columns = ['order_id', 'order_item_id']

if all(col in df_clean.columns for col in key_columns):
    duplicate_rows = df_clean.duplicated(subset=key_columns).sum()
    print(f"Số hàng trùng lặp theo [order_id, order_item_id]: {duplicate_rows}")
    if duplicate_rows == 0:
        print("-> ✅ TỐT! Pipeline vẫn duy trì tính toàn vẹn (integrity).")
    else:
        print("-> 🚨 LỖI! Dữ liệu trùng lặp đã bị tạo ra trong quá trình làm sạch.")
else:
    print("-> 🚨 LỖI! Thiếu cột [order_id, order_item_id] để kiểm tra.")
print("=" * 60)
print("--- HOÀN THÀNH TÁI THẨM ĐỊNH ---")

--- 🩺 BẮT ĐẦU TÁI THẨM ĐỊNH (RE-VALIDATION) TRÊN df_clean ---
--- 1. TÁI KIỂM TRA: Dữ liệu bị thiếu (Null Check) ---
Tổng số Nulls còn lại trong toàn bộ DataFrame: 0
-> ✅ TỐT! Đã dọn sạch (impute) tất cả Nulls.
------------------------------------------------------------
--- 2. TÁI KIỂM TRA: Phân phối (Distribution Check) ---
Min thực tế của 'delivery_time_days' (đã giao): 0.5334
-> ✅ TỐT! Đã làm sạch thành công. Không còn giá trị âm (lỗi).
------------------------------------------------------------
--- 3. TÁI KIỂM TRA: Tính duy nhất (Cardinality Check) ---
Số danh mục (TRƯỚC): 73 (không tính Null)
Số danh mục (SAU): 73 (không tính 'unknown')
-> ✅ TỐT! Các lệnh .lower()/.strip() đã hoạt động (hoặc dữ liệu vốn đã sạch).
------------------------------------------------------------
--- 4. TÁI KIỂM TRA: Tính Toàn vẹn của Join (Integrity Check) ---
Số hàng trùng lặp theo [order_id, order_item_id]: 0
-> ✅ TỐT! Pipeline vẫn duy trì tính toàn vẹn (integrity).
--- HOÀN THÀNH TÁI THẨM ĐỊNH ---
